# Importação de bibliotecas

In [1]:
#biblioteca do OpenAI
%pip install openai-whisper
#biblioteca pra salvar a transcrição
%pip install srt
#bibliotecas para extrair áudio do vídeo
%pip install moviepy
%pip install ffmpeg-python


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from moviepy.editor import VideoFileClip
import whisper
import subprocess
import srt

# Extrair vídeo e áudio

In [ ]:
#extraindo áudio do vídeo
#video_path é o caminho do arquivo. O r na frente do caminho pode ser necessário ou não
video_path = r"C:\Users\LuizFernandoDaCostaC\Box\pessoal\Pessoal\Github\Video-Caption\WhatsApp_Video_2024-11-07_at_11.21.24.mp4"
video = VideoFileClip(video_path) #retira o vídeo
audio_path = "audio.wav" #define o caminho do áudio como .wav
video.audio.write_audiofile(audio_path) #extrai o áudio do vídeo

MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [ ]:
#modelo whisper
model = whisper.load_model("small")  # Pode usar o modelo como small, medium, base ou large. Isso define o tamanho da base de dados em linguagem que será usada para treinar o modelo. Quanto maior o tamanho, maior a acurácia da transcrição, mas maior o tempo de processamento.
# transcrição em português
result = model.transcribe(audio_path, language="pt", word_timestamps=True)

# imprimir o resultado da transcrição
print(result["text"])

 Coisa galera, eu estou aqui com o Patrick ou Chico Moedas Vascaíno, se você preferir? Jennifer Miguel Então vamos lá, primeira pergunta, é verdade que quando a pessoa doa, ela pode contrair alguma doença? É falso Falso, isso é mentira Isso aí galera, é falso, pode doar, sem problema você não vai contrair doença Próxima pergunta, é verdade que se você doar numa frequência maior de três meses, você pode contrair algum tipo de problema na saúde? É falso também É falso Não, isso é mentira Homens podem doar de dois em dois meses, mulheres podem doar de três em três meses É aqui a informação, ele sabe tudo, tá? Passou disso, você pode doar sem problema, você não vai ter problema na saúde E agora, nossa última pergunta, é verdade que uma bolsa de sangue você pode salvar até três vidas? Mentira, você pode doar sangue e salvar até quatro vidas É verdade É verdade E verdade também, gabaritaram aqui, vocês sabem tudo, a situação de sangue, hein? Uma bolsa de sangue salva de três a quatro vidas E

# Criar o texto do vídeo

In [ ]:
#função para dividir o texto por palavras e com o timer a cada palavra dita (pra sincronizar o texto com a fala)
def split_text_with_word_timing(segments, max_chars=35): #max_chars junta as divisões por palavras em caracteres máximos pra aparecer a cada vez
    subs = []
    index = 1

    for seg in segments:
        if "words" in seg:  # Check if word-level timestamps are available
            words = seg["words"]
            line = ""
            line_start = None
            for word in words:
                if line_start is None:
                    line_start = word["start"]
                # Append word to line if it doesn't exceed max length
                if len(line) + len(word["word"]) <= max_chars:
                    line += ("" if line == "" else " ") + word["word"]
                else:
                    # Close the line and save as a subtitle
                    line_end = word["start"]
                    subs.append(srt.Subtitle(index=index, start=srt.timedelta(seconds=line_start), end=srt.timedelta(seconds=line_end), content=line))
                    index += 1
                    # Start a new line with the current word
                    line = word["word"]
                    line_start = word["start"]

            # Append the last line in the segment
            if line:
                line_end = seg["end"]
                subs.append(srt.Subtitle(index=index, start=srt.timedelta(seconds=line_start), end=srt.timedelta(seconds=line_end), content=line))
                index += 1
        else:
            # Fallback to segment-level timing if word-level timestamps are missing
            line = seg["text"]
            line_start = seg["start"]
            line_end = seg["end"]
            subs.append(srt.Subtitle(index=index, start=srt.timedelta(seconds=line_start), end=srt.timedelta(seconds=line_end), content=line))
            index += 1

    return subs



# Get the segments from your transcription result
segments = result["segments"]

# Use the function to split the text based on word-level timestamps
subs = split_text_with_word_timing(segments)

# salvar em arquivo srt (legendas)
srt_filename = "output.srt"
with open(srt_filename, "w", encoding="utf-8") as f:
    f.write(srt.compose(subs))

print(f"Subtitles saved to {srt_filename}")

Subtitles saved to output.srt


# Gerar Vídeo

In [ ]:
#código para gerar o vídeo com legenda usando ffmpeg
output_path = "output_video.mp4" #define o caminho como um arquivo de nome output_video em formato mp4

#parâmetros para a personalização e geração da legenda
ffmpeg_command = [
    'ffmpeg',
    '-i', video_path,  # Input video path
    '-vf', f"subtitles={srt_filename}:force_style='Fontname=Arial Black,Fontsize=17,FontWeight=1000,BorderStyle=1,Outline=1,OutlineColour=&H00000080,Shadow=0.2,MarginV=50'",
    '-c:a', 'copy',  # Copy audio without encoding
    "-c:v", "libx264",             # Video codec (H.264)
    "-c:a", "aac",                 # Audio codec (AAC)
    "-preset", "fast",             # Encoding preset
    '-y',  # Automatically overwrite output file -> importante para não dar erro caso já exista um arquivo com o mesmo nome na pasta
    output_path  # Output file path
]

# executa o comando acima
subprocess.run(ffmpeg_command)

print(f"Video saved as {output_path}") #mensagem para confirmar que o vídeo foi salvo com sucesso

Video saved as output_video.mp4
